# Weather Underground API json Data Acquisition

![WU_logo](wu_logo.jpg)

## 1. Introduction

The purpose of this notebook is to illustrate how to acquire .json data from Weather Underground (WU). To convert the .json files to a single .csv, please see the `Weather Underground json to csv Data Parser`. 

This script download data as-is in JSON format, provided a `start date`, `end date` and a `station id`. The second, which will be shown in a second Notebook, parses the data out of the JSON files and store in a format acceptable by our current database schema. 

**Important**: Please read the following section carefully to avoid being IP blocked. There are very strict guidelines on how many requests can be done per minute and per day. 

## 2. API Request Script

In order to use the API, we need to register for a key. The key does NOT give unlimited access. If you are using the free version (`Stratus Developer`), about 1 year (500 calls per day) can be performed, and 10 requests a minute (you can see this information by [logging in the website](https://www.wunderground.com/weather/api/d/) after requesting an API Key, and then selecting `Key Settings` in the upper part of the page. You can also verify on the website how many calls you already performed on a given day and minute on the webpage. 

If you exceed the amount of requests, it will timeout until the following day. Exception to this is their `raindrop` system. For more details on the raindrop system, please see the [Terms of use](https://www.wunderground.com/weather/api/d/terms.html?MR=1). 

On the script version of this notebook, the key is passed as a parameter to run the script rather than loaded from the file, and is not printed or exposed anywhere on the Notebook, so you do not need to worry about sending this notebook around or uploading to a public Github.

The only required information to execute this notebook are on the following 2 blocks, namely the API key path, time window, station id and save path. 

**Important**: This notebook does not apply any restrictions to the **daily** number of calls. You should reason the number of calls as equivalent to the number of days. At the time this notebook was created, the endpoint used associated 1 day worth of data to 1 call. As such, if your free account only allow 500 requests, do not exceed 500 days. According to Wu's Terms of Use, a day is defined as:

> A day will be defined by the 24-hour period from Midnight to Midnight in New York City local time (either EDT or EST depending on the time of year). A minute will be defined by a 60-second period of time on the WUL servers’ internal clock.

The notebook will, however, **apply a restriction to avoid violating the number of calls per minute**, and will sleep `6.1` seconds after every call. This approximately leads to 30 minutes per 1 year of worth of data. 

In [3]:
with open('keys/api.txt', 'r') as f:
    key = f.readline().rstrip()

Next, we specify: 

 * `start date` (inclusive): YYYYMMDD
 * `end date` (inclusive): YYYYMMDD
 * `station ID`: Check on Weather Underground website (e.g. https://www.wunderground.com/cgi-bin/findweather/getForecast?query=pws:KHIKAPOL19) the ID is **KHIKAPOL19**. 
 * `save_path`: Path to the folder where the downloaded readings will be stored (one file per day worth of readings, each containing as name the timestamp of the given day). 

In [9]:
#Will be loaded as parameter when running the script
start_date = '20161106'
end_date   = '20171106''
station_id = 'KHIKAPOL19'
save_path  = "data/KHIKAPOL19/"

The script then uses the API key, start date, end date, station id and save path to:

 * Enumerate all the list of dates in the range, inclusive (create_list_of_dates)
 * Use one date at a time to create the URL for request worth one day of readings (this is WU's API interface constraint) (create_request_url). 
 * Store the date in an individual JSON file as-is so we can preserve the raw data if we ever need again to extract any additional information years later that was not stored at this point in time in the database by the second script. (download_one_day_readings).

In [5]:
import datetime
def create_list_of_dates(start_date,end_date):
    '''Creates a list of dates based on start and end date, inclusive for both dates. Requestes uses one date at a time.'''
    start = datetime.datetime.strptime(start_date, "%Y%m%d")
    end = datetime.datetime.strptime(end_date, "%Y%m%d")
    dates_list = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days+1)]
    return dates_list

def create_request_url(date,station_id,key):
    '''Creates the URL to obtain the data. This is just a formatting function, it does not send the request itself.'''
    api_url = 'http://api.wunderground.com/api/'
    key = key+'/'
    date = 'history_' + date + '/'
    station = 'q/pws:'+station_id+'.json'
    return (api_url+key+date+station)

import urllib.request
import shutil
def download_one_day_readings(date,url,save_path):
    '''Saves to specified path a file with the date as name downloaded from the formatted url.'''
    with urllib.request.urlopen(url) as response, open(save_path+date+'.json', 'wb') as out_file:
        shutil.copyfileobj(response, out_file)

    

Using the functions should retrieve the desired data in raw-format:

In [10]:
import time
dates_list = create_list_of_dates(start_date,end_date)
total = len(dates_list)
progress = 1
for date in dates_list:
    datestr = date.strftime("%Y%m%d")
    url = create_request_url(datestr,station_id,key)
    print('Downloaded: ',progress,'/',total,'(',date.strftime("%m/%d/%Y"),')')
    time.sleep(6.1) #Only 10 calls per minute per API constraint on our free plan.
    download_one_day_readings(datestr,url,save_path)
    progress+=1 

Downloaded:  1 / 1 ( 11/06/2016 )
